In [1]:
import numpy as np
from as_pysrc.svp_cmds import alignSnds, calc_f0_swipe
from as_pysrc.svp_cmds.transpose_f0 import transpose_f0
from scipy.io import wavfile, loadmat
import os
import glob
from shutil import copyfile
import sys
print(sys.path)

['', '/Users/robinson/Dropbox/anasynth/_code', '/Users/robinson/Dropbox/anasynth/_code/as_pysrc', '/Users/robinson/Dropbox/anasynth/_code/tf_seq2seq', '/u/formes/share/packages/anaconda3/envs/DeepLearning/lib/python36.zip', '/u/formes/share/packages/anaconda3/envs/DeepLearning/lib/python3.6', '/u/formes/share/packages/anaconda3/envs/DeepLearning/lib/python3.6/lib-dynload', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages', '/Users/robinson/Dropbox/anasynth/_code/seq2seq', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/as_pysrc_utils_poly-0.2-py3.6-macosx-10.7-x86_64.egg', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/as_pysrc_utils_levinson-0.1-py3.6-macosx-10.7-x86_64.egg', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/utils_bspline-1.0.0-py3.6-macosx-10.7-x86_64.egg', '/Users/robinson/Dropbox/anasynth/_code/anaconda/lib/python3.6/site-packages/opt_congrad_sc_c-1.2.2

<h3>Remember: Make syllable F0 CSV files for use during WAV conversion using tf_seq2seq_data_processing_test_input.ipynb

In [2]:
# def alignSnds(sndfile_to_be_aligned, outfile, temporal_positions,
#               winsize,  svpcmd=svp_executable, shape=False,
#               transients=False, oversamp=8, vudem=3, transdet_minren=None, verbose=False) :
#     '''
#     use supervp to align sndfile_to_be_aligned to sndfile by means of stretching
#     the first to align to the second.
#     '''
    
'''
# I need a way to link the transformed syllable f0 back to it's source file and position in that file, 
so I can assemble the sequence of transformed f0 contours for a particular phrase

# the problem with test data is that the syllables are a subset of all syllables across all files in the dataset, 
# so not all syllables of a particular phrase will be present.. This makes it difficult to infer only on test data 
in order to test the conversion.

# I could 'cheat', and just infer on the whole dataset (including data i trained on), to see what the conversion is 
like - DONE
# Or I could separate off some complete phrases to test on later (random shuffle), and train on the rest. - TODO 
# would be good to do both, to see the difference it makes! 


# for each wavfile...

# get list of syllables, and make list of true/false to indicate voiced/unvoiced

# make list of before-times (all the start and end times of all voiced and unvoiced parts) - this is just what's in 
the mat file (syll_time) - you prob don't want the final value

# make list of end-times (all the start and end times of all voiced and unvoiced parts)
    # make list of durations of each voiced syllable from new f0 contour, and duration of unvoiced syllable from 
    original file (handle missing files too..):
    # for each syllable in a file:
      # if syll is voiced
          # if file is present:
            # add NEW f0 contour length to dur-list
          # else add original 'before-time' syll length
      # else if unvoiced
          # add before-times unvoiced syll length to dur-list

# make a new list for the final end-times
# for all elements in dur-list
  # add end time to end-times-list as sum of all elements currently in the list, plus next duration in the list - 
  cumsum
          


'''

"\n# I need a way to link the transformed syllable f0 back to it's source file and position in that file, \nso I can assemble the sequence of transformed f0 contours for a particular phrase\n\n# the problem with test data is that the syllables are a subset of all syllables across all files in the dataset, \n# so not all syllables of a particular phrase will be present.. This makes it difficult to infer only on test data \nin order to test the conversion.\n\n# I could 'cheat', and just infer on the whole dataset (including data i trained on), to see what the conversion is \nlike - DONE\n# Or I could separate off some complete phrases to test on later (random shuffle), and train on the rest. - TODO \n# would be good to do both, to see the difference it makes! \n\n\n# for each wavfile...\n\n# get list of syllables, and make list of true/false to indicate voiced/unvoiced\n\n# make list of before-times (all the start and end times of all voiced and unvoiced parts) - this is just what's in \

<h3>Generate before and after syllable timecode csv files 

In [4]:
# path to directory for test_log.txt
csv_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006/f0_raw_syllable'
# Olivia2006.e01.p01.i00.s01_s e1.csv
csv_input_file_extension = '.csv'

# path to predicted F0 CSV input files directory
syll_input_directory_path = '/Users/robinson/Downloads/data/pred/20180628_151243/syllables'
# Olivia2006.e01.p01.i00.s01_s e1.csv
syll_input_file_extension = '.csv'

# path to MAT input files directory
mat_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006'
# Olivia2006.e01.p01.i00.mat
mat_input_file_extension = '.mat'

# path to WAV input files directory
wav_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO'
# Olivia2006.e01.p01.i00.1.wav
wav_input_file_extension = '.wav'

# define input file root - common to all filetypes
input_file_root = 'Olivia2006'

# set window step time 
step_s = 0.005 # 5ms, which I assume is what was used to sample the file

# define output directory and files
output_directory = os.path.join(wav_input_directory_path, 'timestretch')
if not os.path.exists(output_directory):
    os.mkdir(output_directory)


# define phrase range
phrase_from = 1
phrase_to = 10
# define source and target emotion ranges
source_emotion_from = 1
source_emotion_to = 8
# define source and target intensity ranges
source_intensity_from = 0
source_intensity_to = 0

# open test_log.txt and split into list of lines
f_test_log = open(os.path.join(csv_input_directory_path, 'out', 'test_log.txt'), 'r')
test_log = f_test_log.read().split('\n')
# print(test_log)

# output filenames
filename_wav_files = 'wav_files.txt'
filename_syll_time = 'syll_time.txt'
filename_new_syll_time = 'syll_time_new.txt'
filename_syll_vals_min = 'syll_vals_min.txt'
# open output files in subdirectory of input files directory (must create manually)
f_wav_files = open(os.path.join(output_directory, filename_wav_files), 'w')
f_syll_time = open(os.path.join(output_directory, filename_syll_time), 'w')
f_new_syll_time = open(os.path.join(output_directory, filename_new_syll_time), 'w')
f_syll_vals_min = open(os.path.join(output_directory, filename_syll_vals_min), 'w')

# for each wavfile that we want to treat..
# for each phrase
for p in range(phrase_from, phrase_to + 1):
    # for each source emotion
    for e_s in range(source_emotion_from, source_emotion_to + 1):
        # for each source intensity
        for i_s in range(source_intensity_from, source_intensity_to + 1):
            # build the source file path
            filename_base = ''.join([input_file_root, 
                                        '.e', format(e_s, '02d'),
                                        '.p', format(p, '02d'),
                                        '.i', format(i_s, '02d')])
            # to handle the number at the end, get all files beginning with name (there should only be one)
            glob_filepath = os.path.join(wav_input_directory_path, filename_base + '.*')
            wav_filepath_list = glob.glob(glob_filepath)
            wav_filepath = wav_filepath_list[0]
            # print(wav_filepath)
            _, wav_filename = os.path.split(wav_filepath)
            # print('*********************')
            # print('******** WAV ********')
            # print('wav_filename) ', wav_filename)

            # load corresponding mat file
            mat_filename = ''.join([filename_base, mat_input_file_extension])
            mat_filepath = os.path.join(mat_input_directory_path, mat_filename)
            # print(mat_filepath)
            mat_dict = loadmat(mat_filepath)
            
            # get list of syllables
            syll_label = mat_dict['syll_label']
            syll_label = syll_label.reshape((syll_label.shape[1],))
            
            # make list of before-times (all the start and end times of all voiced and unvoiced parts) - this is just what's in 
            # the mat file (syll_time) - you prob don't want the final value. Also, syllable times are not 
            # consecutive - one syllable ends, then there's a gap, then the next begins. So you have to handle the 
            # start and end times of both
            # get list of syllable start/end times
            syll_time = mat_dict['syll_time']
            # print(syll_time.shape)
            # print(syll_time)
            # syll_time = syll_time.reshape((syll_time.shape[1], syll_time.shape[0]))  # wrong!
            syll_time = syll_time.T # much better
            # print(syll_time.shape)
            # print(syll_time)
            # print(syll_time.shape) # (11, 2)
            # print('syll_time = ', syll_time)
            # print(type(syll_time)) # <class 'numpy.ndarray'>
            
            # make list of true/false to indicate voiced(true)/unvoiced(false)
            # [a if C else b for i in items]
            voiced = [True if '#' not in syll[0] else False for i, syll in enumerate(syll_label)]
            # print('voiced ', voiced)
            
            # make new_syll_time list as flattened version of original start and end times, to modify later
            # new_syll_time = list(syll_time.flatten(order='C'))
            # print('syll_time = ', new_syll_time)
            new_syll_time = [0.0] + [x for x in syll_time[:, 1]]  # just get second column, plus add a 0.0 at the start
            # print(new_syll_time)
            
            # create empty list to store all syll_vals, to later calculate the minimum f0 in the voiced regions
            syll_vals_all = []
            # for each syllable in the wav file
            for all_syll_counter, syll in enumerate(syll_label):
                # print('******** SYL ********')
                # if syllable is unvoiced, skip to next syllable
                if not voiced[all_syll_counter]:
                    # print('!!! unvoiced')
                    continue
                    
                # build the filename of the syllable
                filename_syll = ''.join([filename_base, '.s', format(all_syll_counter, '02d'), '_', syll[0], 
                                         str(all_syll_counter), csv_input_file_extension])
                # print('filename_syll ', filename_syll)
                
                # check if syll file is in test_log.txt
                # syll file is not empty, so:
                if filename_syll in test_log:
                    # get duration of original syllable contour
                    syll_dur_orig = syll_time[all_syll_counter, 1] - syll_time[all_syll_counter, 0]
                    # print('syll_dur_orig = ', syll_dur_orig)
                    
                    # get duration of new syllable contour
                    # get number of lines from file
                    fsyll = open(os.path.join(syll_input_directory_path, filename_syll), 'r')
                    syll_vals = fsyll.read().strip().split('\n')
                    # print('syll_vals=', syll_vals)
                    num_lines = len(syll_vals)
                    # print('num_lines= ', num_lines)
                    
                    # multiply by window step size to get time, and add half a window
                    syll_dur_new = (num_lines * step_s) + step_s
                    # print('syll_dur_new = ', syll_dur_new)
                    
                    # calculate the diff (will be positive if new duration is larger)
                    diff = syll_dur_new - syll_dur_orig
                    # print('diff = ', diff)
                    # print('********')
                    
                    # append syll_vals to end of list, to later calculate the minimum f0 in the voiced regions
                    # print(syll_vals)
                    syll_vals_all += syll_vals
                    # print('syll_vals_all ', syll_vals_all)
                    
                # syll file is either missing or empty, so make diff = 0
                else:
                    diff = 0
                    # print('!!! file missing: ', filename_syll)
                    # print('diff = ', diff)
                
                # print('new_syll_time BEFORE UPDATE', new_syll_time)
                # run through the new_syll_time list, adding the (positive/negative) diff to end time of syll, 
                # and start and end times of all subsequent syllables
                # new_syll_time = [time if i-1 < all_syll_counter * 2 else time + diff for i, time in enumerate(
                #     new_syll_time)]
                new_syll_time = [time if i-1 < all_syll_counter else time + diff for i, time in enumerate(
                    new_syll_time)]
                # print('new_syll_time AFTER UPDATE = ', new_syll_time)
                
            
            # convert syll_time to a list
            # syll_time = [val for i, val in enumerate(syll_time)]  
            # syll_time = list(syll_time.flatten(order='C')) # don't need both columns, it just duplicates the times
            syll_time = [0.0] + [x for x in syll_time[:, 1]]  # just get second column, plus add a 0.0 at the start
            # print(syll_time)
            # append syll_time, new_syll_time to outfiles            
            [f_syll_time.write(str(val) + ',') if i < len(syll_time) - 1 else f_syll_time.write(str(val)) for i, 
                                                                                                          val in enumerate(syll_time)]
            f_syll_time.write('\n')
            
            
            
            # [f_new_syll_time.write(str(val) + ',') for val in new_syll_time]
            [f_new_syll_time.write(str(val) + ',') if i < len(new_syll_time) - 1 else f_new_syll_time.write(str(val)) for i, 
                                                                                                          val in enumerate(new_syll_time)]
            f_new_syll_time.write('\n')
            # print to stdout to check
            # [sys.stdout.write(str(val) + ',') for val in new_syll_time]
            # sys.stdout.write('\n')
            
            # print wav file name to file
            f_wav_files.write(wav_filename + '\n')
            
            # convert syll_vals_all to numpy array of integers, get minimum and write to text file
            # print(syll_vals_all)
            syll_vals_all = [int(x) for x in syll_vals_all]
            syll_vals_all = np.array(syll_vals_all)
            # print(syll_vals_all)
            syll_vals_min = syll_vals_all.min()
            # print(syll_vals_min) 
            # if the value is 50 or over the write to file, otherwise default to 50 (eliminates erroneous low values)
            if syll_vals_min >= 50:
                f_syll_vals_min.write(str(syll_vals_min) + '\n')
            else:
                f_syll_vals_min.write(str(50) + '\n')
                

# close the files         
f_test_log.close()
f_wav_files.close()
f_syll_time.close()
f_new_syll_time.close()
f_syll_vals_min.close()

print('done')

done


<h3>Read in syllable timecode csv files and perform timestretch on WAV files

In [5]:
'''
# def alignSnds(sndfile_to_be_aligned, outfile, temporal_positions,
#               winsize,  svpcmd=svp_executable, shape=False,
#               transients=False, oversamp=8, vudem=3, transdet_minren=None, verbose=False) :
#     use supervp to align sndfile_to_be_aligned to sndfile by means of stretching
#     the first to align to the second.

# time stretch whole source soundfile
# param: sndfile_to_be_aligned: source soundfile from olivia2006 dataset
# param: outfile: output file
# param: temporal_positions: list of tuples of time before, time after

shares vars with above code, so run all
'''

# bug: before times has duplicates in it.. 'syll_time.txt'

# open files and split into list of lines
f_wav_files = open(os.path.join(output_directory, filename_wav_files), 'r')
wav_files = f_wav_files.read().split('\n')
f_syll_time = open(os.path.join(output_directory, filename_syll_time), 'r')
syll_times = f_syll_time.read().split('\n')
f_new_syll_time = open(os.path.join(output_directory, filename_new_syll_time), 'r')
new_syll_times = f_new_syll_time.read().split('\n')
f_syll_vals_min = open(os.path.join(output_directory, filename_syll_vals_min), 'r')
syll_vals_min = f_syll_vals_min.read().split('\n')

# debug
# wav_files = ['Olivia2006.e01.p01.i00.1.wav']
# print(wav_files)

# for each file in enumerate(wav_files)
for i, wav_file in enumerate(wav_files):
    # skip if wav_file is empty:
    if not wav_file:
        continue
#     build the full path for sndfile_to_be_aligned
    wav_file_path = os.path.join(wav_input_directory_path, wav_file)
    # print('wav_file_path ', wav_file_path)
#     get before times and after times from corresponding row of the two syll_time files, and put in lists
    before_times = syll_times[i].split(',')
    before_times = [float(x) for x in before_times if x]
    # print('before_times ', before_times)
    end_times = new_syll_times[i].split(',')
    end_times = [float(x) for x in end_times if x]
    # print('end_times ', end_times)
#     make temporal_positions list of tuples from corresponding items from each list
#     temporal_positions = list(zip(before_times, end_times))

    # loop through zipped list backwards, removing any tuples that have identical elements
    # this is to solve RuntimeError("do not support alignment with zero size segments.")
    # length = len(temporal_positions)
    # # print(length)
    # for j in range(length - 1, -1, -1):
    #     print(j)
    #     print(temporal_positions[j])
    #     print(temporal_positions[j][0])
    #     print(temporal_positions[j][1])
    #     if temporal_positions[j][0] == temporal_positions[j][1]:
    #         del temporal_positions[j]
    #         print('del ', j)

    # [del temporal_positions[i] if temporal_positions[i][0] == temporal_positions[i][1] for i, x in enumerate(temporal_positions)]
    # print(temporal_positions)

    # convert cleaned list of tuples to list of two-element arrays
    temporal_positions = [np.array(x) for x in zip(before_times, end_times)]
    # print('temporal_positions ', temporal_positions)
    # print('len ', len(temporal_positions))

#     make outfile name from infile name
#     print('wav_file ', wav_file)
    wav_file_out, ext = os.path.splitext(wav_file)
    wav_file_out = wav_file_out + '.OUT' + ext
    # print('wav_file_out ', wav_file_out)
    wav_file_out_path = os.path.join(output_directory, wav_file_out)
#     make winsize
#     winsize = 4 * int(syll_vals_min[i])
    winsize = 800 # works much better
    # print('i ', i)
    # print('winsize ', winsize)
#     call alignSnds with above params: shape=True, winsize = 4 x f0_min of unvoiced regions of THIS (source) wav file
    alignSnds(wav_file_path, wav_file_out_path, temporal_positions, winsize, shape=True, verbose=True)
    # print('******************')

print('done')

07/02/2018 10:59:34 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD4729.par -M800 -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e01.p01.i00.1.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p01.i00.1.OUT.wav


07/02/2018 10:59:36 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD4729.par -M800 -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e02.p01.i00.62.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e02.p01.i00.62.OUT.wav


07/02/2018 10:59:36 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD4729.par -M800 -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e03.p01.i00.122.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e03.p01.i00.122.OUT.wav


07/02/2018 10:59:37 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD4729.par -M800 -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e04.p01.i00.182.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e04.p01.i00.182.OUT.wav


07/02/2018 10:59:37 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD4729.par -M800 -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e05.p01.i00.242.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e05.p01.i00.242.OUT.wav


...

07/02/2018 11:00:14 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD4729.par -M800 -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e05.p10.i00.298.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e05.p10.i00.298.OUT.wav


07/02/2018 11:00:15 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD4729.par -M800 -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e06.p10.i00.360.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e06.p10.i00.360.OUT.wav


07/02/2018 11:00:15 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD4729.par -M800 -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e07.p10.i00.421.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e07.p10.i00.421.OUT.wav


07/02/2018 11:00:16 : alignSnds::INFO::run: supervp -P0 -Afft -Z -sync_pc 1 -D/tmp/alignD4729.par -M800 -oversamp 8 -S /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/Olivia2006.e08.p10.i00.481.wav -shape 1 -vu_f0lim 1,5 -Vuf -4 -vu_dem 3 /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p10.i00.481.OUT.wav


done


<h3>Read in time stretched WAV files, extract f0 of each, and output result SDIF files

In [6]:
'''
def calc_f0_swipe(infile, outfile, f0min=50, f0max=450, time_step=0.005,
                  harmonicity_threshold=None,
                  use_spline_interp=False, ana_freq_limit=12500, music_mode=False,
                   remove_low_harm_f0=True, channel=0, verbose=False):

    use swipe with given parameters to create the f0 analysis of infile and store the result in an sdif file

    raises RuntimeError in case of errors

'''

# input directory is former output_directory
wav_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO'
input_directory_path = os.path.join(wav_input_directory_path, 'timestretch')
# input_directory_path = wav_input_directory_path  #debug
input_directory = os.fsencode(input_directory_path)

# define new output directory and files
output_directory_path = os.path.join(wav_input_directory_path, 'timestretch_f0extract')
# output_directory_path = os.path.join(wav_input_directory_path, 'original_f0extract')  #debug
if not os.path.exists(output_directory_path):
    os.mkdir(output_directory_path)

# for each wav file in directory
for file in os.listdir(input_directory):
    # get filename
    filename = os.fsdecode(file)
    # get filepath and decode from bytes to string
    # filepath = os.path.abspath(file).decode("utf-8") 
    filepath = os.path.join(input_directory_path, filename)
    print(filepath)
    
    if filename.endswith('.wav'):
        # build outfile sdif filename
        sdif_file_out, ext = os.path.splitext(filename)
        sdif_file_out = sdif_file_out + '.sdif'
        sdif_file_out_path = os.path.join(output_directory_path, sdif_file_out)
        
        # extract f0 and write to sdif
        calc_f0_swipe(filepath, sdif_file_out_path)


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/.DS_Store
/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p01.i00.1.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p02.i00.8.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p03.i00.14.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p04.i00.20.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p05.i00.26.OUT.wav


...

/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p07.i00.463.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p08.i00.469.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p09.i00.475.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p10.i00.481.OUT.wav


/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/syll_time.txt
/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/syll_time_new.txt
/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/syll_vals_min.txt
/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/wav_files.txt


<h3>Read in timestretched wav, timestretched f0 sdif, and target f0 array, and output transposed f0 WAV file

In [7]:
'''
i need to:

piece together all f0 syllable files into one continuous sequence, including unvoiced sections (get durations from 
syll_time), and put into an array


call f0_transpose to convert, passing in timestretched wav, timestretched f0 sdif, and target f0 array
as well as outfile wav file name, and winsize scalar

'''

# for each 
# get list 
# path to CSV input files directory
csv_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006/f0_raw_syllable'
# Olivia2006.e01.p01.i00.s01_s e.csv
csv_input_file_extension = '.csv'

# path to predicted F0 CSV input files directory
syll_input_directory_path = '/Users/robinson/Downloads/data/pred/20180627_155521/syllables'
# Olivia2006.e01.p01.i00.s01_s e1.csv
syll_input_file_extension = '.csv'

# path to MAT input files directory
mat_input_directory_path = '/Users/robinson/Dropbox/anasynth/_data/emoVC/Olivia2006'
# Olivia2006.e01.p01.i00.mat
mat_input_file_extension = '.mat'

# path to WAV input files directory
raw_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/'
# Olivia2006.e01.p01.i00.1.wav
raw_input_file_extension = '.wav'

# path to timestretched WAV input files directory
wav_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch'
# Olivia2006.e01.p01.i00.1.wav
wav_input_file_extension = '.wav'

# path to SDIF f0 input files directory
sdif_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch_f0extract'
# Olivia2006.e01.p01.i00.mat
sdif_input_file_extension = '.sdif'

# define input file root - common to all filetypes
input_file_root = 'Olivia2006'

# define output directory and files
output_directory = os.path.join(raw_input_directory_path, 'f0transpose')
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
    
# open test_log.txt and split into list of lines
f_test_log = open(os.path.join(csv_input_directory_path, 'out', 'test_log.txt'), 'r')
test_log = f_test_log.read().split('\n')

# define phrase range
phrase_from = 1
phrase_to = 10  #10
# define source and target emotion ranges
source_emotion_from = 1
source_emotion_to = 8  #8
# define source and target intensity ranges
source_intensity_from = 0
source_intensity_to = 0

# set sample rate 
step_s = 0.005 # 5ms, which I assume is what was used to sample the file


# for each wavfile that we want to treat..
# for each phrase
for p in range(phrase_from, phrase_to + 1):
    # for each source emotion
    for e_s in range(source_emotion_from, source_emotion_to + 1):
        # for each source intensity
        for i_s in range(source_intensity_from, source_intensity_to + 1):
            # build the source file path
            filename_base = ''.join([input_file_root, 
                                        '.e', format(e_s, '02d'),
                                        '.p', format(p, '02d'),
                                        '.i', format(i_s, '02d')])
            
            # open the mat file 
            mat_filename = ''.join([filename_base, mat_input_file_extension])
            mat_filepath = os.path.join(mat_input_directory_path, mat_filename)
            mat_dict = loadmat(mat_filepath)
            
            # get list of syllables
            syll_label = mat_dict['syll_label']
            syll_label = syll_label.reshape((syll_label.shape[1],))
            
            # get syll_time from dict and transpose to column
            syll_time = mat_dict['syll_time'].T
            # print('syll_time ', syll_time)
            
            # make boolean list of voiced/unvoiced
            voiced = [True if '#' not in syll[0] else False for i, syll in enumerate(syll_label)]
            # print('voiced ', voiced)
            
            # create list to hold sequence of voiced and unvoiced f0 contour values
            all_contours = []
            
            # for each syllable in the wav file
            for all_syll_counter, syll in enumerate(syll_label):
                # print('******** SYL ********')
                # build the filename of the syllable
                filename_syll = ''.join([filename_base, '.s', format(all_syll_counter, '02d'), '_', syll[0],
                                         str(all_syll_counter), csv_input_file_extension])
                
                # if syllable is unvoiced, or if syll file is not in test_log.txt (i.e. it's missing or empty), 
                # get the length of the unvoiced syllable and add zeroes to all_contours list
                if not voiced[all_syll_counter] or filename_syll not in test_log:
                    # print('!!! unvoiced')
                    time = syll_time[all_syll_counter, 1] - syll_time[all_syll_counter, 0]
                    # print('time ', time)
                    num_zeroes = int(time // step_s)
                    # print('num_zeroes ', num_zeroes)
                    all_contours += [0 for _ in range(num_zeroes)]
                # if syllable is voiced, add its contents to the all_contours list
                else:
                    # build the source file path
                    source_file_path = os.path.join(syll_input_directory_path, filename_syll)
                    
                    # load the source file and extract vars
                    source_f0_raw = np.loadtxt(source_file_path, dtype='int')
                    # print('shape ', source_f0_raw.shape)
                    
                    # iterate over the numpy array, adding items to the all_contours list
                    all_contours += [source_f0_raw[x] for x in range(source_f0_raw.shape[0])]
            
            # convert into a single array to pass to f0_transpose method - not sure if needed..
            all_times = [x * step_s for x in range(len(all_contours))]
            all_times_contours = list(zip(all_times, all_contours))
            
            # print('*** length = ', len(all_times_contours))
            for i in range(len(all_times_contours) - 1, 0, -1):
                # print(i)
                if all_times_contours[i][1] == 0:
                    del all_times_contours[i]

            all_times_contours = np.array(all_times_contours)
            # .reshape(-1, 2)
            # print(all_times_contours)
            
            # build timestretched input wav file name and path - use glob to handle number           
            wav_glob_filepath = os.path.join(wav_input_directory_path, filename_base + '.*')
            # print('wav_glob_filepath ', wav_glob_filepath)
            wav_filepath_list = glob.glob(wav_glob_filepath)
            # print('wav_filepath_list ', wav_filepath_list)
            wav_filepath = wav_filepath_list[0] 
            print('wav_filepath ', wav_filepath)
            _, wav_filename = os.path.split(wav_filepath)
            wav_filename_noext, _ = os.path.splitext(wav_filename)
            
            
            # build outfile wav file name and path
            wav_filename_out = ''.join([wav_filename_noext, '.COL', wav_input_file_extension])
            wav_filename_out_path = os.path.join(output_directory, wav_filename_out)
            
            # build sdif file name
            sdif_glob_filepath = os.path.join(sdif_input_directory_path, filename_base + '.*')
            # print('sdif_glob_filepath ', sdif_glob_filepath)
            sdif_filepath_list = glob.glob(sdif_glob_filepath)
            # print('sdif_filepath_list ', sdif_filepath_list)
            sdif_filepath = sdif_filepath_list[0] 
            # print('sdif_filepath ', sdif_filepath)
            
            # set winsize scalar
            winsize = 800  # worked before, but dunno why
            
            # call transpose_f0
            transpose_f0(wav_filepath, sdif_filepath, all_times_contours, wav_filename_out_path, winsize)


wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e01.p01.i00.1.OUT.wav


wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e02.p01.i00.62.OUT.wav


wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e03.p01.i00.122.OUT.wav


wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e04.p01.i00.182.OUT.wav


wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e05.p01.i00.242.OUT.wav


...

wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e04.p10.i00.236.OUT.wav


wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e05.p10.i00.298.OUT.wav


wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e06.p10.i00.360.OUT.wav


wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e07.p10.i00.421.OUT.wav


wav_filepath  /Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch/Olivia2006.e08.p10.i00.481.OUT.wav


In [9]:
# copy original source files to directory to compare

# path to raw WAV files directory
raw_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/'
# Olivia2006.e01.p01.i00.1.wav

# path to timestretched WAV files directory
wav_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/timestretch'

# path to timestretched f0 tranformed files directory
joy_input_directory_path = '/Users/robinson/Downloads/data/Olivia2006/Olivia2006_AUDIO/f0transpose'

# define input file root - common to all filetypes
input_file_root = 'Olivia2006'

# define output directory and files
output_directory = os.path.join(raw_input_directory_path, 'compare')
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

# define phrase range
phrase_from = 1
phrase_to = 10  #10
# define source and target emotion ranges
source_emotion_from = 1
source_emotion_to = 8  #8
# define source and target intensity ranges
source_intensity_from = 0
source_intensity_to = 0

# for each wavfile that we want to treat..
# for each phrase
for p in range(phrase_from, phrase_to + 1):
    # for each source emotion
    for e_s in range(source_emotion_from, source_emotion_to + 1):
        # for each source intensity
        for i_s in range(source_intensity_from, source_intensity_to + 1):
            # build the source file path
            filename_base = ''.join([input_file_root, 
                                        '.e', format(e_s, '02d'),
                                        '.p', format(p, '02d'),
                                        '.i', format(i_s, '02d')])
            
            # build input wav file name and path - use glob to handle number           
            wav_glob_filepath = os.path.join(raw_input_directory_path, filename_base + '.*')
            # print('wav_glob_filepath ', wav_glob_filepath)
            wav_filepath_list = glob.glob(wav_glob_filepath)
            # print('wav_filepath_list ', wav_filepath_list)
            wav_filepath = wav_filepath_list[0] 
            # print('wav_filepath ', wav_filepath)
            _, wav_filename = os.path.split(wav_filepath)
            
            # copy the raw file to the output directory
            copyfile(wav_filepath, os.path.join(output_directory, wav_filename))
            
            
# copy all timestretched wav files to compare directory
wav_input_directory = os.fsencode(wav_input_directory_path)
# for each wav file in directory
for file in os.listdir(wav_input_directory):
    # get filename
    filename = os.fsdecode(file)
    if filename.endswith('.wav'):
        # get filepath and decode from bytes to string
        # filepath = os.path.abspath(file).decode("utf-8") 
        filepath = os.path.join(wav_input_directory_path, filename)
        
        # copy the timestretched wav file to the output directory
        #copyfile(filepath, os.path.join(output_directory, filename))
    
    
# copy all f0 transformed wav files to compare directory
joy_input_directory = os.fsencode(joy_input_directory_path)
# for each wav file in directory
for file in os.listdir(joy_input_directory):
    # get filename
    filename = os.fsdecode(file)
    if filename.endswith('.wav'):
        # get filepath and decode from bytes to string
        # filepath = os.path.abspath(file).decode("utf-8") 
        filepath = os.path.join(joy_input_directory_path, filename)
        
        # copy the timestretched wav file to the output directory
        copyfile(filepath, os.path.join(output_directory, filename))

print('done')

done
